In [ ]:
!pip install - r requirements.txt
#安裝顯示表格套件
!pip install pandas

In [ ]:
##API初始&登入
import pfcf
from pfcf.pfcf import *
from pfcf.product import *
from pfcf.trade.dlogic import *
from typing import List
#匯入顯示資料套件
import pandas as pd   
from IPython.display import display,clear_output 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
#初始API
api = Pfcf()
#錯誤事件處理
api.on_error = lambda error: print(f'api錯誤 {error}')
#系統登入
loginresponse = api.login(
    "url", "帳號", "密碼", '憑證檔名', '憑證密碼')#url,帳號,密碼,憑證檔名,憑證密碼
#顯示登入結果
print(loginresponse) 
print(f"登入旗標{api.login_status_flag}")
print(f"交易帳號{api.get_accounts()}") 
##初始資料
reply_data = [ ['','','','','','','','','','','','','','','','','','','','','']]
reply_dd = pd.DataFrame(reply_data) 
reply_dd.columns=['分公司','帳號','網路流水序號','委託時間','委託書號','子帳',
            '商品類別','商品代碼','買賣別','價格別','委託價格','委託數量','未成交數量',
            '成交數量','刪除數量','下單方式','開倉別','異動時間','委託狀態','委託狀態碼','下單序號'] 
match_data = [ ['','','','','','','','','','','','','','','','']]
match_dd = pd.DataFrame(match_data) 
match_dd.columns=['分公司','帳號','網路流水序號','成交時間','委託書號','子帳',
            '商品類別','商品代碼','買賣別','成交價格','成交口數','成交序號',
            '成交價1','成交價2','備註','異動時間']  
actno = '使用帳號'# 使用帳號

In [ ]:
#訂閱回報接收範例  
def on_reply(orderreply: DOrderReply): 
    i=len(reply_dd)+1 
    reply_dd.loc[i, '分公司'] = orderreply.brokerid
    reply_dd.loc[i, '帳號'] = orderreply.investoracno
    reply_dd.loc[i, '網路流水序號'] = orderreply.networkid
    reply_dd.loc[i, '委託時間'] = orderreply.ordertime
    reply_dd.loc[i, '委託書號'] = orderreply.orderno
    reply_dd.loc[i, '子帳'] = orderreply.subact
    reply_dd.loc[i, '商品類別'] = orderreply.productkind
    reply_dd.loc[i, '商品代碼'] = orderreply.productid
    reply_dd.loc[i, '買賣別'] = orderreply.bs
    reply_dd.loc[i, '價格別'] = orderreply.ordertype
    reply_dd.loc[i, '委託價格'] = orderreply.price
    reply_dd.loc[i, '委託數量'] = orderreply.orderqty
    reply_dd.loc[i, '未成交數量'] = orderreply.nomatchqty
    reply_dd.loc[i, '成交數量'] = orderreply.matchqty
    reply_dd.loc[i, '刪除數量'] = orderreply.delqty
    reply_dd.loc[i, '下單方式'] = orderreply.ordercondition
    reply_dd.loc[i, '開倉別'] = orderreply.opencloseflag
    reply_dd.loc[i, '異動時間'] = orderreply.mdate
    reply_dd.loc[i, '委託狀態'] = orderreply.orderstatus
    reply_dd.loc[i, '委託狀態碼'] = orderreply.statuscode
    reply_dd.loc[i, '開倉別'] = orderreply.opencloseflag
    reply_dd.loc[i, '下單序號'] = orderreply.seq 
    clear_output(wait=True)  
    display(reply_dd)  
    display(match_dd)  
def on_match(matchreply: DMatchReply): 
    i=len(reply_dd)+1 
    match_dd.loc[i, '分公司'] = matchreply.brokerid
    match_dd.loc[i, '帳號'] = matchreply.investoracno
    match_dd.loc[i, '網路流水序號'] = matchreply.networkid
    match_dd.loc[i, '成交時間'] = matchreply.matchtime
    match_dd.loc[i, '委託書號'] = matchreply.orderno
    match_dd.loc[i, '子帳'] = matchreply.subact
    match_dd.loc[i, '商品類別'] = matchreply.productkind
    match_dd.loc[i, '商品代碼'] = matchreply.productid
    match_dd.loc[i, '買賣別'] = matchreply.bs
    match_dd.loc[i, '成交價格'] = matchreply.matchprice
    match_dd.loc[i, '成交口數'] = matchreply.matchqty
    match_dd.loc[i, '成交序號'] = matchreply.matchseq
    match_dd.loc[i, '成交價1'] = matchreply.matchpricefoot1
    match_dd.loc[i, '成交價2'] = matchreply.matchpricefoot2
    match_dd.loc[i, '備註'] = matchreply.note
    match_dd.loc[i, '異動時間'] = matchreply.mdate 
    clear_output(wait=True)  
    display(reply_dd)  
    display(match_dd)  
api.dtrade.on_reply=on_reply
api.dtrade.on_match=on_match 
print('訂閱完畢')

In [ ]:
#下單 
order = DOrderObject()
order.actno = actno#帳號
order.note = "ordertest"#備註
order.subactno = ""#子帳
order.productid = "TXFF5"#商品代號
order.bs = "B"#買賣別B/S
order.ordertype = "M"#L:限價 M:市價 P:範圍市價
order.price = 0#委託價格
order.orderqty = 5#數量
order.ordercondition = "R"##委託種類 I:IOC R:ROD F:FOK
order.opencloseflag = ""##開倉別 str 0:新倉 1:平倉 空白:自動
order.dtrade="N"## Y:當沖 N:非當沖
orderresponse=api.dtrade.order(order) 
print(f" 是否成功 {orderresponse.issend} 下單序號 {orderresponse.seq} 錯誤代碼 {orderresponse.errorcode} 錯誤訊息 {orderresponse.errormsg}")

In [ ]:
#刪單 
replace=DReplaceObject()
replace.replacetype="4"#4:取消, 5: 減量, m:改價"
replace.actno = actno#帳號
replace.orderno="P1z8o"#委託書號
replaceresponse=api.dtrade.replace_order(replace)
print(f" 是否成功 {replaceresponse.issend} 下單序號 {replaceresponse.seq} 錯誤代碼 {replaceresponse.errorcode} 錯誤訊息 {replaceresponse.errormsg}")

In [ ]:
#減量 
replace=DReplaceObject()
replace.replacetype="5"#4:取消, 5: 減量, m:改價"
replace.actno =actno#帳號
replace.orderno="P1yk8"#委託書號
replace.orderqty = 1#減量口數
replaceresponse=api.dtrade.replace_order(replace)
print(f" 是否成功 {replaceresponse.issend} 下單序號 {replaceresponse.seq} 錯誤代碼 {replaceresponse.errorcode} 錯誤訊息 {replaceresponse.errormsg}")

In [ ]:
#改價 
replace=DReplaceObject()
replace.replacetype="m"#4:取消, 5: 減量, m:改價"
replace.actno =actno#帳號
replace.orderno="P1yk8"#委託書號
replace.ordertype="L"#L:限價 M:市價 P:範圍市價
replace.ordercondition="R"#委託種類 I:IOC R:ROD F:FOK
replace.price=22300#委託價格
replaceresponse=api.dtrade.replace_order(replace)
print(f" 是否成功 {replaceresponse.issend} 下單序號 {replaceresponse.seq} 錯誤代碼 {replaceresponse.errorcode} 錯誤訊息 {replaceresponse.errormsg}")

In [ ]:
##委回查詢範例 
query_reply_response=api.dtrade.query_reply(actno,500,"","","","")#帳號,筆數,網路流水序號起,,網路流水序號迄,委託時間起,委託時間迄
print(query_reply_response)
if query_reply_response.ok == True: 
        i=0
        for orderreply in query_reply_response.data:  
            reply_dd.loc[i,'分公司'] = orderreply.brokerid
            reply_dd.loc[i,'帳號'] = orderreply.investoracno
            reply_dd.loc[i,'網路流水序號'] = orderreply.networkid
            reply_dd.loc[i,'委託時間'] = orderreply.ordertime
            reply_dd.loc[i,'委託書號'] = orderreply.orderno
            reply_dd.loc[i,'子帳'] = orderreply.subact
            reply_dd.loc[i,'商品類別'] = orderreply.productkind
            reply_dd.loc[i,'商品代碼'] = orderreply.productid
            reply_dd.loc[i,'買賣別'] = orderreply.bs
            reply_dd.loc[i,'價格別'] = orderreply.ordertype
            reply_dd.loc[i,'委託價格'] = orderreply.price
            reply_dd.loc[i,'委託數量'] = orderreply.orderqty
            reply_dd.loc[i,'未成交數量'] = orderreply.nomatchqty
            reply_dd.loc[i,'成交數量'] = orderreply.matchqty
            reply_dd.loc[i,'刪除數量'] = orderreply.delqty
            reply_dd.loc[i,'下單方式'] = orderreply.ordercondition
            reply_dd.loc[i,'開倉別'] = orderreply.opencloseflag
            reply_dd.loc[i,'異動時間'] = orderreply.mdate
            reply_dd.loc[i,'委託狀態'] = orderreply.orderstatus
            reply_dd.loc[i,'委託狀態碼'] = orderreply.statuscode
            reply_dd.loc[i,'開倉別'] = orderreply.opencloseflag
            reply_dd.loc[i,'下單序號'] = orderreply.seq       
            i=i+1
        clear_output(wait=True)  
        display(reply_dd)               
else:
    print("查詢發生錯誤"+query_reply_response.error)          


In [ ]:
##成回查詢範例 
query_match_response=api.dtrade.query_match(actno,500,"","","","")#帳號,筆數,網路流水序號起,,網路流水序號迄,成交時間起,成交時間迄
if query_match_response.ok == True: 
        for matchreply in query_match_response.data:  
            match_dd.loc[i,'分公司'] = matchreply.brokerid
            match_dd.loc[i,'帳號'] = matchreply.investoracno
            match_dd.loc[i,'網路流水序號'] = matchreply.networkid
            match_dd.loc[i,'成交時間'] = matchreply.matchtime
            match_dd.loc[i,'委託書號'] = matchreply.orderno
            match_dd.loc[i,'子帳'] = matchreply.subact
            match_dd.loc[i,'商品類別'] = matchreply.productkind
            match_dd.loc[i,'商品代碼'] = matchreply.productid
            match_dd.loc[i,'買賣別'] = matchreply.bs
            match_dd.loc[i,'成交價格'] = matchreply.matchprice
            match_dd.loc[i,'成交口數'] = matchreply.matchqty
            match_dd.loc[i,'成交序號'] = matchreply.matchseq
            match_dd.loc[i,'成交價1'] = matchreply.matchpricefoot1
            match_dd.loc[i,'成交價2'] = matchreply.matchpricefoot2
            match_dd.loc[i,'備註'] = matchreply.note
            match_dd.loc[i,'異動時間'] = matchreply.mdate
            i=i+1 
        clear_output(wait=True)  
        display(match_dd)         
else:
    print("查詢發生錯誤"+query_match_response.error)       

In [ ]:
#系統登出
api.logout()
print('登出完畢')
print(f"登入旗標{api.login_status_flag}")
